In [1]:
import pickle
from gensim import corpora, models
import numpy as np

In [2]:
with open("../../topic_models_crop/data2/50-topic-res-all.txt", "rb") as fp:
    img_by_topic = pickle.load(fp)


In [3]:
# get full list of images
img_list = np.load('../../topic_models_crop/data/image_list_yfcc100m_output_100k_crop.npy')

In [4]:
# get images in topic animals with posteriors above 0.5
topic_animals_img_indices = [img_by_topic[35][i][0] for i in range(0, 5000) if img_by_topic[35][i][1] >= 0.5]
# get urls of topic animal images
animal_image_urls = img_list[topic_animals_img_indices]

np.save('data/img_list_animals',animal_image_urls)

In [5]:
# get binary yfcc100 vector
yfcc_binary = np.load('../../topic_models_crop/data/one_hot_yfcc100m.npy')

In [6]:
# get binary vectors corresponding to topic animals
animals_binary = yfcc_binary[topic_animals_img_indices]

np.save('data/animals_binary',animals_binary)

In [7]:
X = np.load('data/animals_binary.npy')

In [8]:
num_topics = [15]
num_words = 4096 # Number of top features to be displayed per topic
num_images = X.shape[0]
print 'shape of data is:',X.shape

shape of data is: (186, 4096)


In [9]:
# get corpus corresponding to images in topic animals
corpus = [[(j, X[i, j]) for j in range(num_words) if X[i, j]==1] for i in range(num_images)]

In [10]:
corpora.MmCorpus.serialize('data/corpus_animals.mm', corpus)


In [11]:
# Load corpus
corpus = corpora.MmCorpus('data/corpus_animals.mm')

In [12]:
animal_image_urls = np.load('data/img_list_animals.npy')

In [13]:
for K in num_topics:
    print K
    # Create the Topic Model
    model_name = str(K) + '-topics.model'
    lda = models.ldamodel.LdaModel(corpus, num_topics = K,  minimum_probability=0)
    lda.save('results/' + model_name)

    # Get topic for each image
    img_by_topic = [[] for _ in range(K)]
    for i in range(num_images):
        sorted_topics = sorted(lda.get_document_topics(corpus[i]), key=lambda x:x[1])
        for item in sorted_topics:
            img_by_topic[item[0]].append((i,item[1]))


    for j in range(K):
        img_by_topic[j].sort(key=lambda x: -x[1])

    with open("results/" + str(K) + "-topic-res.txt", "wb") as fp:
        pickle.dump(img_by_topic, fp)

15
